In [1]:
import sys
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
plt.style.use('seaborn-v0_8-paper')
pt = 1./72.27
fig_width = 239 * pt
aspect_ratio = (1 + 5 ** 0.5) / 2
plt.rcParams["figure.figsize"] = (fig_width, fig_width/aspect_ratio)
plt.rcParams["savefig.bbox"] = "tight"
sns.set_theme(style='whitegrid', context='paper', rc={'axes.facecolor': 'white', 'figure.figsize': (fig_width, fig_width/aspect_ratio)})

In [3]:
column_names = ["Device", "Num GPUs", "GPU-TDP", "CPU-TDP", "GPU-Flops", "Carbon Rate", "Time", "Energy-kJ"]
gpu_data = [
    ["P100", 1, 250, 200, 6700, 8.5, 2321, 889],
    ["P100", 2, 250, 200, 6700, 9.1, 1396, 635],
    ["V100", 1, 250, 200, 14000, 19, 1494, 1316],
    ["V100", 2, 250, 200, 14000, 20, 1190, 1194],
    ["V100", 4, 250, 200, 14000, 23, 917, 916],
    ["V100", 8, 250, 200, 14000, 28, 926, 944],
    ["A100", 1, 400, 200, 18000, 87, 1405, 2100],
    ["A100", 2, 400, 200, 18000, 93, 926, 1427],
    ["A100", 4, 400, 200, 18000, 106, 841, 1320],
    ["A100", 8, 400, 200, 18000, 131, 838, 1325],
]

gpu_df = pd.DataFrame(gpu_data, columns=column_names)
gpu_df

,Device,Num GPUs,GPU-TDP,CPU-TDP,GPU-Flops,Carbon Rate,Time,Energy-kJ
0,P100,1,250,200,6700,8.5,2321,889
1,P100,2,250,200,6700,9.1,1396,635
2,V100,1,250,200,14000,19.0,1494,1316
3,V100,2,250,200,14000,20.0,1190,1194
4,V100,4,250,200,14000,23.0,917,916
5,V100,8,250,200,14000,28.0,926,944
6,A100,1,400,200,18000,87.0,1405,2100
7,A100,2,400,200,18000,93.0,926,1427
8,A100,4,400,200,18000,106.0,841,1320
9,A100,8,400,200,18000,131.0,838,1325


In [4]:
gpu_df["TDP"] = (gpu_df["Num GPUs"] * gpu_df["GPU-TDP"]) + gpu_df["CPU-TDP"]
gpu_df["Flops"] = (gpu_df["Num GPUs"] * gpu_df["GPU-Flops"])

gpu_df["Energy"] = gpu_df["Energy-kJ"] * 1000
gpu_df["Avg. Power"] = gpu_df["Energy"] / gpu_df["Time"]
gpu_df["Energy_Cost"] = (gpu_df["Energy"] + (gpu_df["Time"] * gpu_df["TDP"]))/2
gpu_df["Carbon_Cost"] = (gpu_df["Energy"] * 53/1000) + (gpu_df["Time"] * gpu_df["Carbon Rate"]/60)
gpu_df["Perf_Cost"] = gpu_df["Time"] * gpu_df["Flops"]
gpu_df["Energy_Cost_Norm"] = gpu_df["Energy_Cost"] / gpu_df["Energy_Cost"].min()
gpu_df["Carbon_Cost_Norm"] = gpu_df["Carbon_Cost"] / gpu_df["Carbon_Cost"].min()
gpu_df["Perf_Cost_Norm"] = gpu_df["Perf_Cost"] / gpu_df["Perf_Cost"].min()
gpu_df

,Device,Num GPUs,GPU-TDP,CPU-TDP,GPU-Flops,Carbon Rate,Time,Energy-kJ,TDP,Flops,Energy,Avg. Power,Energy_Cost,Carbon_Cost,Perf_Cost,Energy_Cost_Norm,Carbon_Cost_Norm,Perf_Cost_Norm
0,P100,1,250,200,6700,8.5,2321,889,450,6700,889000,383.024558,966725.0,47445.808333,15550700,1.199262,1.400956,1.000000
1,P100,2,250,200,6700,9.1,1396,635,700,13400,635000,454.871060,806100.0,33866.726667,18706400,1.000000,1.000000,1.202930
2,V100,1,250,200,14000,19.0,1494,1316,450,14000,1316000,880.856760,994150.0,70221.100000,20916000,1.233284,2.073454,1.345020
3,V100,2,250,200,14000,20.0,1190,1194,700,28000,1194000,1003.361345,1013500.0,63678.666667,33320000,1.257288,1.880272,2.142669
4,V100,4,250,200,14000,23.0,917,916,1200,56000,916000,998.909487,1008200.0,48899.516667,51352000,1.250713,1.443881,3.302231
5,V100,8,250,200,14000,28.0,926,944,2200,112000,944000,1019.438445,1490600.0,50464.133333,103712000,1.849150,1.490080,6.669282
6,A100,1,400,200,18000,87.0,1405,2100,600,18000,2100000,1494.661922,1471500.0,113337.250000,25290000,1.825456,3.346566,1.626293
7,A100,2,400,200,18000,93.0,926,1427,1000,36000,1427000,1541.036717,1176500.0,77066.300000,33336000,1.459496,2.275576,2.143698
8,A100,4,400,200,18000,106.0,841,1320,1800,72000,1320000,1569.560048,1416900.0,71445.766667,60552000,1.757722,2.109615,3.893844
9,A100,8,400,200,18000,131.0,838,1325,3400,144000,1325000,1581.145585,2087100.0,72054.633333,120672000,2.589133,2.127594,7.759908
